In [1]:
import copy
import numpy as np
from tqdm import tqdm
import warnings
from numpy.random import normal, randint
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
import re
%load_ext autoreload
%autoreload 2

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


from inn import *
from datasetall.datautils import *
from clfs.clfutils import *

# load dataset utils
dataset = InnDataSet("heloc", test_size=0.3)
X, y, df, columns, ordinal_features, discrete_features, continuous_features, feat_var_map = load_dataset_utils(
    dataset)
dataset_name = "heloc"

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]
[WARNING] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.
 [lazy_loader.py _load]


In [2]:
print(X.shape, y.shape)

(9871, 21) (9871, 1)


In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
# these X test is for testing multiplicity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
seed = 0
clf0 = MLPClassifier(hidden_layer_sizes=(20, 20), random_state=seed, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)
clf0.fit(X_train, y_train)

MLPClassifier(batch_size=64, hidden_layer_sizes=(20, 20),
              learning_rate='adaptive', learning_rate_init=0.02,
              random_state=0)

In [3]:
clf = MLPClassifier(hidden_layer_sizes=(15, 15), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7330 test_accuracy with a std of 0.0146
0.7322 test_f1_macro with a std of 0.0145


In [4]:
clf = MLPClassifier(hidden_layer_sizes=(20, 15), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7383 test_accuracy with a std of 0.0092
0.7379 test_f1_macro with a std of 0.0094


In [5]:
clf = MLPClassifier(hidden_layer_sizes=(20, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7405 test_accuracy with a std of 0.0082
0.7390 test_f1_macro with a std of 0.0084


In [6]:
clf = MLPClassifier(hidden_layer_sizes=(30, 20), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7335 test_accuracy with a std of 0.0068
0.7316 test_f1_macro with a std of 0.0064


In [7]:
clf = MLPClassifier(hidden_layer_sizes=(30, 25), learning_rate='adaptive', batch_size=64, learning_rate_init=0.02)

scoring = ['accuracy', 'f1_macro']
scores = cross_validate(clf, X_train, y_train, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.4f %s with a std of %0.4f" % (scores[name].mean(), name, scores[name].std()))

0.7362 test_accuracy with a std of 0.0085
0.7360 test_f1_macro with a std of 0.0084


In [8]:
clfs = []
total_num = 150
seed = 0
scores_acc = []
scores_compl = []
sizes = [(15,15),(20,15),(20,20),(30,20),(30,25)]
sizes_score = [0, 0.25, 0.5, 0.75, 1]
for i in tqdm(range(0, total_num)):
    X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X_train, y_train, test_size=0.2, random_state=2*seed+15)
    sizes_idx = np.random.randint(0,5)
    this_clf = MLPClassifier(hidden_layer_sizes=sizes[sizes_idx], random_state=3*seed+21, learning_rate='adaptive', batch_size=64, learning_rate_init=0.02).fit(X_train_0, y_train_0)
    seed += 12
    if len(np.unique(this_clf.predict(X_test))) == 1:
        continue
    scores_acc.append(accuracy_score(this_clf.predict(X_test), y_test))
    scores_compl.append(sizes_score[sizes_idx])
    #if accuracy_score(y_test, this_clf.predict(X_test)) < 0.73 or accuracy_score(y_test, this_clf.predict(X_test)) > 0.77:
    #    continue
    clfs.append(this_clf)
#clfs.append(clf0)
print(len(clfs))

100%|██████████| 150/150 [03:45<00:00,  1.50s/it]

150


In [10]:
from methods import Datasets, ExperimentsForOneModelSize
d_idxs = np.arange(0, len(X_test))
np.random.seed(12345)
if len(X_test) >= 500:
    d_idxs = np.random.randint(0, len(X_test) - 1, 500)
X_test_run = pd.DataFrame(data=X_test.values[d_idxs], columns=X_test.columns)
y_test_run = pd.DataFrame(data=y_test.values[d_idxs], columns=y_test.columns)

d = Datasets(X_train, y_train, X_test_run, y_test_run)
for i, m in enumerate(clfs):
    scores_acc[i] = accuracy_score(y_test_run, m.predict(X_test_run))

In [17]:
exp_10 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=10, dataset_name="heloc", seed=11432)
exp_10.eval()

100%|██████████| 5/5 [04:56<00:00, 59.39s/it]


In [19]:
exp_10.eval_ours()
exp_10.print_results()

100%|██████████| 5/5 [00:08<00:00,  1.63s/it]

model size = 10, mean, single model acc: 0.7090799999999999
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.709 |    0.495 |   0.943 |   0.943 |   0.657 |     0.000 |       0.022 |                    |              |
| mvcva   | 0.709 |    0.495 |   0.943 |   0.309 |   1.000 |     0.335 |       0.022 |                    |              |
| ours    | 0.712 |    0.504 |   0.499 |   0.499 |   1.000 |     0.000 |       0.164 |              0.145 |        0.983 |
| ours-a  | 0.726 |    0.485 |   0.357 |   0.357 |   1.000 |     0.000 |       0.000 |              0.000 |        0.943 |
| ours-s  | 0.710 |    0.608 |   0.462 |   0.462 |   1.000 |     0.000 |       

In [13]:
exp_20 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=20, dataset_name="heloc", seed=3333)
exp_20.eval()

100%|██████████| 5/5 [11:18<00:00, 135.61s/it]


In [14]:
exp_20.eval_ours()
exp_20.print_results()

100%|██████████| 5/5 [00:13<00:00,  2.68s/it]

model size = 20, mean, single model acc: 0.71036
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.717 |    0.488 |   0.940 |   0.940 |   0.626 |     0.000 |       0.009 |                    |              |
| mvcva   | 0.717 |    0.488 |   0.940 |   0.230 |   1.000 |     0.372 |       0.009 |                    |              |
| ours    | 0.716 |    0.466 |   0.460 |   0.460 |   1.000 |     0.000 |       0.105 |              0.095 |        0.984 |
| ours-a  | 0.728 |    0.432 |   0.361 |   0.361 |   1.000 |     0.000 |       0.018 |              0.016 |        0.950 |
| ours-s  | 0.711 |    0.551 |   0.420 |   0.420 |   1.000 |     0.000 |       0.078 |    

In [15]:
exp_30 = ExperimentsForOneModelSize(d, clfs, scores_acc, scores_compl, model_size=30, dataset_name="heloc", seed=3333)
exp_30.eval()

100%|██████████| 5/5 [16:50<00:00, 202.18s/it]


In [16]:
exp_30.eval_ours()
exp_30.print_results()

100%|██████████| 5/5 [00:20<00:00,  4.05s/it]

model size = 30, mean, single model acc: 0.7102933333333334
+---------+-------+----------+---------+---------+---------+-----------+-------------+--------------------+--------------+
| name    |   acc |   m simp |   m len |   c len |   c val |   abstain |   multi_ans |   multi_same_class |   same_as_mv |
+=========+=======+==========+=========+=========+=========+===========+=============+====================+==============+
| mvca    | 0.718 |    0.512 |   0.940 |   0.940 |   0.620 |     0.000 |       0.009 |                    |              |
| mvcva   | 0.718 |    0.512 |   0.940 |   0.205 |   1.000 |     0.411 |       0.009 |                    |              |
| ours    | 0.716 |    0.499 |   0.456 |   0.456 |   1.000 |     0.000 |       0.078 |              0.072 |        0.982 |
| ours-a  | 0.729 |    0.406 |   0.353 |   0.353 |   1.000 |     0.000 |       0.000 |              0.000 |        0.946 |
| ours-s  | 0.712 |    0.518 |   0.445 |   0.445 |   1.000 |     0.000 |       